<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_0_tuning_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0   1021      0 --:--:-- --:--:-- --:--:--  1021
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-0.test.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,3,cracking sensations in my joints
3,5,fast and slow heartbeat
4,5,increased heart rate


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_pubmed_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.456011,4.801585,0.230895,11:00


In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.save_encoder('pubmed_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.922778,3.646910,0.402367,01:09


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.146024,3.224728,0.467456,01:26


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.595240,3.010381,0.489941,03:14


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.312857,2.772238,0.540828,03:17
1,2.147611,2.669141,0.553846,03:25


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.042233,2.555311,0.570414,03:22
1,1.846292,2.466280,0.583432,03:31


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.772295,2.388847,0.595266,03:15
1,1.585190,2.326642,0.608284,03:39


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.553138,2.268492,0.603550,03:10
1,1.463210,2.232801,0.608284,03:34


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.422895,2.237030,0.613018,03:15
1,1.399768,2.188566,0.622485,03:33


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.320471,2.175039,0.629586,03:14
1,1.314083,2.129364,0.614201,03:36
2,1.266564,2.101860,0.636686,03:16
3,1.151678,2.126555,0.634320,03:14


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.188476,2.161037,0.635503,03:27
1,1.227790,2.167259,0.635503,03:10
2,1.158809,2.130969,0.635503,03:26
3,1.053355,2.078233,0.640237,03:28


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.035989,2.123456,0.636686,03:28
1,1.115023,2.210546,0.633136,03:10
2,1.047107,2.124221,0.634320,03:30
3,1.011073,2.156649,0.634320,03:20


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.074412,2.150791,0.636686,03:34
1,1.099807,2.184929,0.641420,03:27
2,1.061084,2.201993,0.634320,03:20
3,0.977975,2.172127,0.643787,03:21


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.947238,2.202555,0.640237,03:25
1,1.016514,2.245541,0.634320,03:20
2,1.048654,2.218162,0.639053,03:26
3,0.951389,2.173524,0.644970,03:51


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.929622,2.211712,0.642604,03:46
1,1.071171,2.271442,0.639053,03:24
2,0.976035,2.166731,0.631953,03:44
3,0.860735,2.134399,0.629586,03:42


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.958871,2.253240,0.622485,03:33
1,1.025088,2.189805,0.646154,03:53
2,1.016411,2.189776,0.647337,03:56
3,0.932819,2.146877,0.654438,03:36


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-pubmed')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [43]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[194, 289, 73, 551, 5, 8, 8, 8, 8, 8, 8, 8, 674, 5, 817, 817, 135, 27, 27, 491, 491, 211, 211, 211, 31, 584, 817, 34, 289, 289, 289, 289, 289, 289, 289, 37, 515, 44, 269, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 100, 100, 100, 100, 100, 53, 817, 58, 61, 71, 71, 491, 76, 76, 117, 741, 741, 741, 817, 817, 817, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223, 289, 289, 289, 182, 182, 80, 80, 80, 80, 80, 80, 80, 80, 80, 88, 89, 427, 268, 785, 817, 92, 94, 245, 431, 94, 94, 94, 94, 94, 817, 591, 98, 492, 98, 100, 834, 100, 100, 100, 100, 100, 100, 100, 100, 291, 214, 88, 88, 108, 209, 111, 289, 289, 289, 265, 111, 289, 117, 120, 817, 817, 41, 125, 817, 630, 125, 125, 125, 125, 125, 125, 125, 778, 125, 127, 129, 131, 745, 142, 142, 144, 318, 37, 817, 265, 434, 344, 354, 158, 178, 165, 166, 359, 166, 101, 172, 289, 172, 172, 172, 172, 172, 742, 289, 289, 182, 182, 470, 289, 183, 183, 183, 183, 183, 183, 183, 18

In [44]:
correct_num = 0

for i in range(0, 867):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

477
0.5501730103806228
